# Part I. ETL Pipeline for Pre-Processing the Files
#### Import Python packages 

In [72]:
# Import Python packages

import csv
import glob
import os


#### Creating list of filepaths to process original event csv data files

In [73]:
# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [74]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line)
            

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [75]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [76]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [77]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [78]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

Our SELECT query would be:

> SELECT artist, song_title, song_length FROM sessions WHERE session_id=338 and item_in_session=4

So, our table for this query would basically consists of the columns mentioned in the above query with the primary key composed of the columns in the  WHERE clause. We go ahead and create the this 'sessions' table:

In [79]:
try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS sessions (session_id int, item_in_session int,
            artist text, song_title text, song_length float,
            PRIMARY KEY(session_id, item_in_session))
    """)
except Exception as e:
    print(e)

In [80]:
# Insert the data in the table

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    sessions_insert_query_template = "INSERT INTO sessions (session_id, item_in_session, artist, " \
                                     "song_title, song_length) VALUES (%s, %s, %s, %s, %s)"

    for line in csvreader:
        artist, _, _, item_in_session, _, song_length, _, _, session_id, song_title, _ = line
        session.execute(sessions_insert_query_template, (int(session_id), int(item_in_session),
                                                         artist, song_title, float(song_length)))

#### Do a SELECT to verify that the data have been inserted into sessions table

In [81]:
select_session_query = "SELECT artist, song_title, song_length FROM sessions WHERE session_id=338 and item_in_session=4"
try:
    rows = session.execute(select_session_query)
    for row in rows:
        print(f"Artist: {row.artist}, Song title: {row.song_title}, Song length: {row.song_length}")
except Exception as e:
    print(e)




Artist: Faithless, Song title: Music Matters (Mark Knight Dub), Song length: 495.30731201171875


## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

Our SELECT query would be:

> SELECT artist, song_title, item_in_session, first_name, last_name FROM user_sessions
> WHERE user_id=10 AND session_id=182

So, our table for this query would basically consists of the columns mentioned in the above query with the primary key
composed of the columns in the WHERE clause plus there could be multiple songs in one user session and we want
the result to be sorted by item in session and hence we also add it in the partition key component of the Primary Key.
<br><br>
We go ahead and create the this 'user_sessions' table:

In [82]:
try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS user_sessions (user_id int, session_id int,
            artist text, song_title text, item_in_session int, first_name text, last_name text,
            PRIMARY KEY((user_id, session_id), item_in_session))
    """)
except Exception as e:
    print(e)

In [83]:
# Insert the data in the table

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    sessions_insert_query_template = "INSERT INTO user_sessions (user_id, session_id, artist, song_title, " \
                                     "item_in_session, first_name, last_name) VALUES (%s, %s, %s, %s, %s, %s, %s)"

    for line in csvreader:
        artist, first_name, _, item_in_session, last_name, _, _, _, session_id, song_title, user_id = line
        session.execute(sessions_insert_query_template, (int(user_id), int(session_id), artist, song_title,
                                                         int(item_in_session), first_name, last_name))

#### Do a SELECT to verify that the data have been inserted into user_sessions table

In [84]:
select_user_session_query = "SELECT artist, song_title, item_in_session, first_name, last_name" \
                       " FROM user_sessions WHERE user_id=10 AND session_id=182"
try:
    rows = session.execute(select_user_session_query)
    for row in rows:
        print(f"Artist: {row.artist}, Song title: {row.song_title}, Item in Session: {row.item_in_session}, "
              f"User Name: {row.first_name + ' ' + row.last_name}")
except Exception as e:
    print(e)

                    

Artist: Down To The Bone, Song title: Keep On Keepin' On, Item in Session: 0, User Name: Sylvie Cruz
Artist: Three Drives, Song title: Greece 2000, Item in Session: 1, User Name: Sylvie Cruz
Artist: Sebastien Tellier, Song title: Kilometer, Item in Session: 2, User Name: Sylvie Cruz
Artist: Lonnie Gordon, Song title: Catch You Baby (Steve Pitron & Max Sanna Radio Edit), Item in Session: 3, User Name: Sylvie Cruz


## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

Our SELECT query would be:

> SELECT first_name, last_name FROM user_songs WHERE song_title='All Hands Against His Own';

So, our table for this query would basically consists of the columns mentioned in the above query with the primary key
composed of the columns in the WHERE clause plus there could be multiple users listening to one song and we don't want
users overwriting each other (as per Cassandra's property) for a song and hence we also add user's identifier 'user_id'
as the partition key component of the Primary Key.
<br><br>
We go ahead and create the this 'user_sessions' table:
                    

In [85]:
try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS user_songs (user_id int, song_title text, first_name text, last_name text,
        PRIMARY KEY(song_title, user_id))
    """)
except Exception as e:
    print(e)


In [86]:
# Insert the data in the table


file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    sessions_insert_query_template = "INSERT INTO user_songs (user_id, song_title, " \
                                     "first_name, last_name) VALUES (%s, %s, %s, %s)"

    for line in csvreader:
        _, first_name, _, _, last_name, _, _, _, _, song_title, user_id = line
        session.execute(sessions_insert_query_template, (int(user_id), song_title,
                                                         first_name, last_name))


#### Do a SELECT to verify that the data have been inserted into user_songs table

In [87]:
select_user_songs_query = "SELECT first_name, last_name FROM user_songs " \
                            "WHERE song_title='All Hands Against His Own'"
try:
    rows = session.execute(select_user_songs_query)
    for row in rows:
        print(f"User Name: {row.first_name + ' ' + row.last_name}")
except Exception as e:
    print(e)

User Name: Jacqueline Lynch
User Name: Tegan Levine
User Name: Sara Johnson


### Drop the tables before closing out the sessions

In [88]:
session.execute("DROP TABLE sessions")

session.execute("DROP TABLE user_sessions")

session.execute("DROP TABLE user_songs")


### Close the session and cluster connection¶

In [89]:
session.shutdown()
cluster.shutdown()